# NTP

[toc]

## Network Time Protocol
Network Time Protocol. This will make your system date match actual date, by synchronizing with network time servers.

本文将介绍在一个集群中选取一台服务器搭建ntp服务器，集群中其他服务器以平滑的方式同步该ntp服务器时间，以保证集群的时间一致性。

集群主机列表：
192.168.153.121   host1.centos
192.168.153.122   host2.centos
192.168.153.123   host3.centos
192.168.153.124   host4.centos
其中主机 192.168.153.121   host1.centos 作为 ntp 服务器主机，其他三台主机作为客户端主机从 host1.centos 自动同步时间。

## ntp服务器配置
从集群中选取一台服务器作为 ntp 服务器，在该主机上进行如下操作：
- 检查 ntp 是否已安装
  ```
  $ yum list installed | grep ntp
  ntp.x86_64                            4.2.6p5-29.el7.centos.2          @base    
  ntpdate.x86_64                        4.2.6p5-29.el7.centos.2          @base   
  ```
- 修改配置文件 /etc/ntp.conf
  ```
  $ vi /etc/ntp.conf

  # 在 /etc/ntp.conf 文件中添加如下配置，授权当前集群服务器网段内主机可以通过该服务器同步时间
  restrict 192.168.153.0 mask 255.255.255.0 nomodify notrap

  # 在 /etc/ntp.conf 文件中添加下面 ntp 服务器地址，配置网络时间服务器地址，网络时间服务器地址官网 https://www.ntppool.org/zh/
  server ntp.tuna.tsinghua.edu.cn iburst
  server time.ustc.edu.cn iburst
  server cn.pool.ntp.org iburst

  # 在 /etc/ntp.conf 文件中添加下面信息，确保断网情况下使用该服务器时间作为标准时间
  server 127.127.1.0
  fudge 127.127.1.0 stratum 10
  ```
- 修改配置文件 /etc/sysconfig/ntpd
  ```
  # 在 /etc/sysconfig/ntpd 配置文件中添加如下信息，实现搭建的 ntp 服务器硬件时间和系统时间一起同步
  SYNC_HWCLOCK=yes
  ```

- 重启 ntp 服务
  ```
  # run ntp daemon service
  $ systemctl restart ntpd.service

  # checkout ntpd is running
  $ ps -ef | grep ntpd
  root        918      1  0 11:07 ?        00:00:00 /usr/sbin/ntpd
  root       1609   1578  0 16:40 pts/0    00:00:00 grep --color=auto ntp
  ```

- 配置 ntp 服务开机自启动
  ```
  # 通过 /etc/rc.local 文件配置开机自启动服务

  # show rc.local info
  $ ls -ahl /etc/rc.local
  lrwxrwxrwx. 1 root root 13 Jul 23 17:33 /etc/rc.local -> rc.d/rc.local

  # /etc/rc.local 是 /etc/rc.d/rc.local 软连接，检查 /etc/rc.d/rc.local 文件权限。
  $ ls -ahl /etc/rc.d/rc.local
  -rw-r--r--. 1 root root 514 Jul 27 10:31 /etc/rc.d/rc.local

  # /etc/rc.d/rc.local 赋执行权限
  $ chmod +x /etc/rc.d/rc.local 
  $ ls -ahl /etc/rc.d/rc.local 
  -rwxr-xr-x. 1 root root 514 Jul 27 10:31 /etc/rc.d/rc.local

  # 添加开机自启动配置，在文件尾行追加服务启动命令
  $ vi /etc/rc.local 

  # 例如追加 ntpd 服务自启动
  $ cat /etc/rc.local
  #!/bin/bash
  ...
  ...
  /usr/sbin/ntpd

  # 重启服务器，检查服务是否可以开机后自启动
  $ shutdown -r now
  $ ps -ef | grep ntp
  root        918      1  0 11:07 ?        00:00:00 /usr/sbin/ntpd
  root       1434   1418  0 11:08 pts/0    00:00:00 grep --color=auto ntp
  ``` 

## 客户端主机配置
配置集群中其他客户端主机，使其从 ntp 服务器同步时间

- 配置定时同步任务
  ```
  # 使用crontab工具配置定时任务
  $ crontab -e

  # 添加以下定时同步配置，客户端主机每5分钟触发一次从ntp服务器同步时间请求
  */5 * * * * /usr/sbin/ntpdate host1.centos
  ```
- 启动 crontab 服务
  ```
  # 执行下面命令，启动crontab服务
  $ systemctl start crond

  # 检查服务是否已启动
  $ ps -ef | grep crond
  root        682      1  0 04:56 ?        00:00:00 /usr/sbin/crond -n
  root       1467   1416  0 05:03 pts/0    00:00:00 grep --color=auto crond
  ```
- 配置 crond.service 服务开机自启动
  ```
  # 运行下面命令配置crond开机自启动
  $ sudo systemctl enable crond.service

  # 重启服务器，检查服务是否可以开机后自启动
  $ shutdown -r now
  $ ps -ef | grep crond
  root        682      1  0 04:56 ?        00:00:00 /usr/sbin/crond -n
  root       1467   1416  0 05:03 pts/0    00:00:00 grep --color=auto crond
  ```

## 测试同步结果
任意选取一台客户端主机，修改时间后，看5分中后是否自动同步时间。
```
# 修改客户端主机时间
$ date -s "2022-7-27 16:50:00"

# 5分钟后检查该客户机是否自动同步时间
$ date
Wed Jul 27 05:12:38 EDT 2022
You have new mail in /var/spool/mail/root
```
